<a href="https://colab.research.google.com/github/whernandezp-hue/PPCDII/blob/main/TRANSFERENCIA_PPCDII_WILLIAMH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se decide cambiar el modelo realizado en la entrega anterior:
El principal motivo por el que se decide hacer el cambio del modelo es que anteriormente se obtuvo una precisión de solo el 59%, lo que en muchos casos puede ser un porcentaje alto y considerable. Pero, en este caso es muy riesgoso trabajar con un modelo que se usará para la toma de decisiones en la compra y venta de diamantes, por lo que se debe buscar un modelo que nos dé´un porcentaje de precisión lo más cercano al 100% posible.

Se hizo la simulación y se obtuvo que el mejor modelo para este caso (prueba) es el modelo Random Forest
---



In [3]:
# 1.Primero se importan las librerías necesarias
import kagglehub
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import plotly.express as px

In [4]:
# 2. Descargamos el dataset directamente desde Kaggle para garantizar que se pueda usar el colab sin importan en qué equipo se corra el código
path = kagglehub.dataset_download("shivam2503/diamonds")
print("Path to dataset files:", path)
df = pd.read_csv(f"{path}/diamonds.csv")


Using Colab cache for faster access to the 'diamonds' dataset.
Path to dataset files: /kaggle/input/diamonds


In [5]:
# 3.En este paso primero limpiamos un poco la BBDD
# Eliminamos filas nulas
df = df.dropna()

# Convertir variables categóricas a dummies
df = pd.get_dummies(df, drop_first=True)

# Variable objetivo = price
X = df.drop("price", axis=1)
y = df["price"]

In [6]:
# 4. Dividir datos en entrenamiento y entrenamiento del modelo Random Forest
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
best_rf = RandomForestRegressor(random_state=42, n_estimators=100)
best_rf.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [7]:
# 5. Evaluación
y_pred = best_rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Error cuadrático medio (MSE): {mse:.4f}")
print(f"R2 Score: {r2:.4f}")

df_resultados = pd.DataFrame({"Reales": y_test.values, "Predichos": y_pred})
print(df_resultados)

fig = px.scatter(df_resultados, x="Reales", y="Predichos",
                 title="Valores Reales vs Predichos (Random Forest - Diamonds)",
                 labels={"Reales": "Valores Reales", "Predichos": "Valores Predichos"})
fig.add_shape(type="line",
              x0=df_resultados["Reales"].min(),
              y0=df_resultados["Reales"].min(),
              x1=df_resultados["Reales"].max(),
              y1=df_resultados["Reales"].max(),
              line=dict(color="red", dash="dash"))
fig.show()


Error cuadrático medio (MSE): 476.9865
R2 Score: 1.0000
       Reales  Predichos
0         559     556.90
1        2201    2200.84
2        1238    1238.00
3        1304    1304.30
4        6901    6899.09
...       ...        ...
10783    1554    1553.83
10784     633     631.04
10785     761     754.94
10786    9836    9836.20
10787    3742    3742.01

[10788 rows x 2 columns]


In [8]:
## 6. Dashboard interactivo
!pip uninstall -y dash jupyter-dash
!pip install dash==2.11.1 jupyter-dash==0.4.2

from dash import dcc, html, Input, Output
from jupyter_dash import JupyterDash

# Traducción de variables para que sea más fácil de entender
traducciones = {
    "carat": "Quilates",
    "depth": "Profundidad",
    "table": "Tabla",
    "x": "Longitud (x)",
    "y": "Anchura (y)",
    "z": "Altura (z)"
}
for col in X.columns:
    if col not in traducciones:
        traducciones[col] = col

# Layout del Dashboard
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("📊 Dashboard Interactivo - Diamantes"),

    html.Label("Selecciona variable independiente (X):"),
    dcc.Dropdown(
        id="x-axis",
        options=[{"label": traducciones[col], "value": col} for col in X.columns],
        value="carat"
    ),

    dcc.Graph(id="scatter-plot"),

    html.H3("Valores Reales vs Predichos"),
    dcc.Graph(
        id="real-vs-pred",
        figure=px.scatter(
            x=y_test,
            y=y_pred,
            labels={"x": "Valores Reales", "y": "Valores Predichos"},
            title="Valores Reales vs Predichos (Random Forest)"
        )
    )
])

# Callback interactivo
@app.callback(
    Output("scatter-plot", "figure"),
    Input("x-axis", "value")
)
def update_scatter(x_col):
    return px.scatter(
        df,
        x=x_col,
        y="price",
        labels={x_col: traducciones.get(x_col, x_col), "price": "Precio"},
        title=f"Precio vs {traducciones.get(x_col, x_col)}",
        opacity=0.6
    )

app.run_server(mode="inline", port=8050)

Found existing installation: dash 2.11.1
Uninstalling dash-2.11.1:
  Successfully uninstalled dash-2.11.1
Found existing installation: jupyter-dash 0.4.2
Uninstalling jupyter-dash-0.4.2:
  Successfully uninstalled jupyter-dash-0.4.2
  Using cached dash-2.11.1-py3-none-any.whl.metadata (11 kB)
  Using cached jupyter_dash-0.4.2-py3-none-any.whl.metadata (3.6 kB)
Using cached dash-2.11.1-py3-none-any.whl (10.4 MB)
Using cached jupyter_dash-0.4.2-py3-none-any.whl (23 kB)


/usr/local/lib/python3.12/dist-packages/dash/dash.py:516: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>